In this notebook we analyze connected rank 4 matroids on 8 elements. We develop theory that can be used to eliminate matroids from consideration. In particular, we check how many rank 3 circuits an elements are contained in.

In [11]:
using Oscar
using Combinatorics
pm = Polymake

include("matroid_realization.jl");
include("reduceIdeals.jl")
include("tscCoordRing.jl");

d4n8 = vec(readlines("d4n8.dat"));

In [15]:
connected_d4n8 = []
for z in 1:length(d4n8)
    Mz = matroid_from_revlex_basis_encoding(d4n8[z], 4, 8)
    if n_connected_components(Mz) == 1
        push!(connected_d4n8,z)
    end
end


***Condition:*** There exists a rank 3 circuit. 

In [16]:
c3 = []
for z in connected_d4n8
    Mz = matroid_from_revlex_basis_encoding(d4n8[z], 4, 8)
    c = [t for t in circuits(Mz) if length(t) == 3]
    if length(c)>0
        push!(c3,z)
    end
end

In [17]:
length(c3)

271

***Counting number of circuits or circuits an element is contained in.*** 

note that hyperplane or circuit always refers to cardinality 4

In [18]:
#function that counts the number of lines/hyperplanes/cirtcuits containing an element of [n]
#for an element n, counts how many lines its in
function in_lines(n,L)
    xn = 0
    for z in 1:length(L)
        if n in L[z]
           xn += 1 
        end
    end
    return xn
end

in_lines (generic function with 1 method)

***No circuits, no hyperplanes***

In [19]:
no_circuits = []
for z in connected_d4n8
    Mz = matroid_from_revlex_basis_encoding(d4n8[z], 4, 8)
    L = [h for h in circuits(Mz) if length(h) == 4]
    ns = [n for n in 1:8 if in_lines(n,L) == 0]
    if length(ns)>0
        push!(no_circuits,z)
    end
end
length(no_circuits)

101

In [20]:
no_hyps = []
for z in connected_d4n8
    Mz = matroid_from_revlex_basis_encoding(d4n8[z], 4, 8)
    L = [h for h in hyperplanes(Mz) if length(h) == 4]
    ns = [n for n in 1:8 if in_lines(n,L) == 0]
    if length(ns)>0
        push!(no_hyps,z)
    end
end
length(no_hyps)

105

I am pretty sure not being in any circuits gives us the same torus factor as not being in any proper hyperplanes...

***One proper hyperplane***

In [21]:
one_hyp = []
for z in connected_d4n8
   Mz = matroid_from_revlex_basis_encoding(d4n8[z], 4, 8)
   L = [h for h in hyperplanes(Mz) if length(h) == 4]
   ns = [n for n in 1:8 if in_lines(n,L) == 1]
   if length(ns)>0
        push!(one_hyp,z)
   end
end
length(one_hyp)

206

***one circuit***

In [22]:
one_circuit = []
for z in connected_d4n8
   Mz = matroid_from_revlex_basis_encoding(d4n8[z], 4, 8)
    L = [h for h in circuits(Mz) if length(h) == 4]
    ns = [n for n in 1:8 if in_lines(n,L) == 1]
    if length(ns)>0
        push!(one_circuit,z)
    end
end
length(one_circuit)

174

In [23]:
issubset(one_circuit,one_hyp)

false

***I don't know if I have a proof for two hyperplanes, haven't been able to completely convince myself***

***Two circuits, $\eta_1,\eta_2$ such that there exists a basis $B$ with $\eta_1\setminus n\subset B$ and $\eta_2\setminus\eta_1 \cap B\neq\emptyset$***

In [24]:
two_circs_prop = []
for z in connected_d4n8
    
    Mz = matroid_from_revlex_basis_encoding(d4n8[z], 4, 8)
    Hyps = [c for c in circuits(Mz) if length(c) == 4]
    ns = [n for n in 1:8 if in_lines(n,Hyps) == 2]
    
    for n in ns
        
        Cn = [c for c in Hyps if n in c]
        bn1 = [b for b in bases(Mz) if issubset(setdiff(Cn[1],[n]),b)]
        int = [intersect(setdiff(Cn[2],[n]),b) for b in bn1]
        if length(int)>0
            push!(two_circs_prop,z)
        end
    end
end
two = unique(two_circs_prop)  
length(two)

327

***Condition***
There exists an element $t$ such that $t$ is contained in 3 circuit hyperplanes 
$\eta_1$, $\eta_2$, $\eta_3$, and there exists a basis $B$ such that $\eta_1\setminus t\subset B$, and we have $B\setminus\eta_1\subset\eta_2, \eta_3$, and $|\eta_i\cap B|>=2$ for $i \in [3]$. (would like something cleaner, but this has been the only guarantee I have found that the second and third equation have a coefficient in the semigroup for a nonzero $x_{i,4}$.)

In [25]:
uu = []
for z in connected_d4n8
    Mz = matroid_from_revlex_basis_encoding(d4n8[z], 4, 8)
    C4 = [c for c in circuits(Mz) if length(c) == 4]
    ns = [n for n in 1:8 if in_lines(n,C4) == 3]
    Bz = bases(Mz)
    for n in ns
        Cn = [setdiff(c,n) for c in C4 if n in c]
        b1 = [b for b in Bz if issubset(Cn[1],b)]
        b2 = [b for b in b1 if length(intersect(b,Cn[2]))>1 && issubset(setdiff(b,Cn[1]),Cn[2])]
        b3 = [b for b in b2 if length(intersect(b,Cn[2]))>1 && !(issubset(setdiff(Cn[2],b),Cn[3]))]
        
        
        if length(b3)>0 
            push!(uu,z)
        end
    end
end
three = unique!(uu)
length(three)

318

In [26]:
K = union(no_circuits,no_hyps,one_circuit, one_hyp,c3,two,three)
length(K)

547

In [27]:
S = setdiff(connected_d4n8,K)
length(S)

45

In [14]:
270 in S #connected matroid with no rank 4 circuits.

true

We now compute the realization spaces for the remaining matroids, excluding 270. We check whether or not the ideal reduces to the 0 ideal, is nonrealizable, or if more work is needed.

In [29]:
zero_ideal = []
not_realizable = []
more_needed = []

for s in setdiff(setdiff(connected_d4n8,K),[270])
    #print(s,"\n")
    Ms = matroid_from_revlex_basis_encoding(d4n8[s],4,8)
    I = matroid_to_reduced_expression(Ms, QQ,2)
    
    if I[1] == [0]
        push!(zero_ideal,s)
        
    elseif (1 in I[1] || -1 in I[1] || I == "Not Realizable 0 in Semigroup")
        
        push!(not_realizable,s)
        
    else
        push!(more_needed,s)
        
    end
end

In [36]:
length(zero_ideal)

19

In [37]:
length(not_realizable)

13

In [38]:
length(more_needed)

12

We look at the ideals corresponding to the realization spaces of the remaining matroids.

In [30]:
for z in more_needed
    print(z,";","\n\n")
    Ms = matroid_from_revlex_basis_encoding(d4n8[z],4,8)
    I = matroid_to_reduced_expression(Ms, QQ,2)
    
    if I isa String 
        print(I,"\n\n")
        
    else 
        print(I[1],"\n\n"," has ", length(I[1])-1," generator(s)","\n\n")
    end
end

73;

fmpq_mpoly[0, x_{1, 3}^2*x_{3, 3} - x_{1, 3}^2 - x_{1, 3}*x_{3, 3}^2 + x_{3, 3}^2]

 has 1 generator(s)

96;

fmpq_mpoly[0, x_{2, 3}^2*x_{3, 3} - x_{2, 3}^2 - x_{2, 3}*x_{3, 3}^2 + x_{2, 3}*x_{3, 3} + x_{2, 3} - x_{3, 3}]

 has 1 generator(s)

100;

fmpq_mpoly[0, x_{2, 3}^2*x_{3, 3} + x_{2, 3}^2 - x_{2, 3}*x_{3, 3}^2 - 2*x_{2, 3}*x_{3, 3} - x_{2, 3} + x_{3, 3}^2 + x_{3, 3}]

 has 1 generator(s)

173;

fmpq_mpoly[0, 2*x_{2, 3}*x_{3, 3} - x_{2, 3} - x_{3, 3}^2 - x_{3, 3} + 1]

 has 1 generator(s)

180;

fmpq_mpoly[0, x_{2, 3}*x_{3, 3} + x_{2, 3} - 2*x_{3, 3}]

 has 1 generator(s)

181;

fmpq_mpoly[0, x_{2, 3}*x_{3, 3} + x_{2, 3} - 2*x_{3, 3}, 2*x_{2, 3} - x_{3, 3} - 1]

 has 2 generator(s)

183;

fmpq_mpoly[0, x_{2, 3}^2 - 1]

 has 1 generator(s)

184;

fmpq_mpoly[0, x_{2, 3}^2 - 1]

 has 1 generator(s)

229;

fmpq_mpoly[0, x_{3, 3}^2 - 3*x_{3, 3} + 1]

 has 1 generator(s)

231;

fmpq_mpoly[0, 3*x_{1, 3}^2 - 3*x_{1, 3} + 1]

 has 1 generator(s)

260;

fmpq_mpoly[0, 3]

 has 1 genera

Analysis of 270

In [5]:
#wierd matroid
M270 = matroid_from_revlex_basis_encoding(d4n8[270], 4, 8)

Matroid of rank 4 on 8 elements

In [6]:
nonbases(M270)

14-element Vector{Vector{Int64}}:
 [1, 2, 3, 4]
 [1, 2, 5, 6]
 [1, 2, 7, 8]
 [1, 3, 5, 7]
 [1, 3, 6, 8]
 [1, 4, 5, 8]
 [1, 4, 6, 7]
 [2, 3, 5, 8]
 [2, 3, 6, 7]
 [2, 4, 5, 7]
 [2, 4, 6, 8]
 [3, 4, 5, 6]
 [3, 4, 7, 8]
 [5, 6, 7, 8]

In [7]:
automorphism_group(M270)

Group([ (1,2)(7,8), (2,3)(6,7), (3,5)(4,6), (1,2)(3,4) ])

In [9]:
describe(automorphism_group(M270))

"(C2 x C2 x C2) : PSL(3,2)"

In [8]:
C = circuits(M270)

14-element Vector{Vector{Int64}}:
 [1, 2, 3, 4]
 [1, 2, 5, 6]
 [1, 2, 7, 8]
 [1, 3, 5, 7]
 [1, 3, 6, 8]
 [1, 4, 5, 8]
 [1, 4, 6, 7]
 [2, 3, 5, 8]
 [2, 3, 6, 7]
 [2, 4, 5, 7]
 [2, 4, 6, 8]
 [3, 4, 5, 6]
 [3, 4, 7, 8]
 [5, 6, 7, 8]

exclusively has rank 3 circuits, each element contained in exactly 7.

In [37]:
is_binary(M270)

true

In [38]:
is_ternary(M270)

false

In [14]:
#we compute the TSC of M270, and its reduced coordinate ring 
#(see tsc CoordinRing.jl and reduceideals.jl for code)

In [13]:
TSC_to_reduced_expression(M270,QQ)[1]

2-element Vector{MPolyDecRingElem{fmpq, fmpq_mpoly}}:
 0
 2

So far I think the conditions I have above will also work for (4,9) matroids thanks to duality and our (3,8) paper. However, I think we will still need to keep thinking up new ones since there are way more (4,9) matroids. I am not sure what to make of the 270 guy.



check out hypersurfaces for smoothness with Jacobian criteria

In [32]:
for z in more_needed
     Ms = matroid_from_revlex_basis_encoding(d4n8[z],4,8)
     I = matroid_to_reduced_expression(Ms, QQ,2)
    if length(I[1]) == 2
        
        f = I[1][2]
        R = parent(f)
        v = vars(f)
        J = union([derivative(f,x) for x in v],[f])
        
          
        Sgens = I[2]
        S = MPolyPowersOfElement(R , Sgens)
        
        Rloc ,iota = localization(R,S)
        Q = ideal(Rloc,J)
        Sat = saturated_ideal(Q)
        print(z," : ", Sat,"\n\n")
        
    else 
        
        print(z," : ","not hypersurface","\n\n")
    end
end
        
    
    

73 : ideal(1)

96 : ideal(1)

100 : ideal(1)

173 : ideal(1)

180 : ideal(1)

181 : not hypersurface

183 : ideal(1)

184 : ideal(1)

229 : ideal(1)

231 : ideal(1)

260 : ideal(3)

269 : ideal(-2)

